# 2 Xử lí dữ liệu
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## 2.1 Cài đặt thư viện và chuẩn bị dữ liệu thô

In [1]:
%pip install matplotlib pandas numpy seaborn wordcloud
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
raw_df = pd.read_csv('Data/GenshinImpact.vi.csv')

In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   post_id                        300 non-null    int64  
 1   text                           289 non-null    object 
 2   post_text                      289 non-null    object 
 3   shared_text                    68 non-null     object 
 4   original_text                  1 non-null      object 
 5   time                           300 non-null    object 
 6   timestamp                      300 non-null    int64  
 7   image                          227 non-null    object 
 8   image_lowquality               300 non-null    object 
 9   images                         300 non-null    object 
 10  images_description             300 non-null    object 
 11  images_lowquality              300 non-null    object 
 12  images_lowquality_description  300 non-null    obj

In [5]:
print(f"Kích cỡ dữ liệu thô : {raw_df.shape[0]} x {raw_df.shape[1]}")

Kích cỡ dữ liệu thô : 300 x 53


## 2.2 Xử lí dữ liệu

### 2.2.1 Xóa các bài viết bị lỗi thu dữ liệu

In [6]:
zero_reaction_rows = raw_df[raw_df['reaction_count'] == 0]
indices_of_zero_reaction_rows = zero_reaction_rows.index
print(indices_of_zero_reaction_rows)

Index([123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       ...
       290, 291, 292, 293, 294, 295, 296, 297, 298, 299],
      dtype='int64', length=177)


Nhận thấy rằng những bài viết từ bài viết có index = 123 trở đi đều bị lỗi. Nên ta lược bỏ các bài viết từ bài viết có index = 123 trở đi.

In [7]:
cooked_df = raw_df.drop(index=range(123, raw_df.index.max() + 1))

In [8]:
cooked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   post_id                        123 non-null    int64  
 1   text                           123 non-null    object 
 2   post_text                      123 non-null    object 
 3   shared_text                    28 non-null     object 
 4   original_text                  0 non-null      object 
 5   time                           123 non-null    object 
 6   timestamp                      123 non-null    int64  
 7   image                          91 non-null     object 
 8   image_lowquality               123 non-null    object 
 9   images                         123 non-null    object 
 10  images_description             123 non-null    object 
 11  images_lowquality              123 non-null    object 
 12  images_lowquality_description  123 non-null    obj

### 2.2.2 Xóa các trường dữ liệu không chứa bất cứ dữ liệu nào

Nhận thấy vẫn còn các trường không chứa dữ liệu gì

In [9]:
cooked_df = cooked_df.dropna(axis=1, how='all')

Lấy các trường dữ liệu cần thiết

### 2.2.3 Lọc ra các trường dữ liệu cần thiết

In [10]:
cooked_df = cooked_df[['post_url', 'post_text', 'time', 'image', 'video', 'comments', 'shares', 'user_id', 'comments_full', 'reactions', 'reaction_count']]

### 2.2.4 Tùy chỉnh dữ liệu cho phù hợp với việc phân tích

Thêm cột số lượng từng cảm xúc riêng

In [11]:
df1 = cooked_df
df1['reactions'] = df1['reactions'].apply(lambda x : dict(eval(x)))
df2 = df1['reactions'].apply(pd.Series)

cooked_df = pd.concat([df1, df2], axis=1).drop('reactions', axis=1)

C:\Users\AD\AppData\Local\Temp\ipykernel_17092\1140268458.py:3: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df2 = df1['reactions'].apply(pd.Series)


Sửa số lượng của các cột cảm xúc không có số lượng cảm xúc thành số lượng là 0 và sửa kiểu dữ liệu

In [12]:
for i in range(0, len(cooked_df['thích'].values), 1):
    if pd.isna(cooked_df['thích'].values[i]):
        cooked_df['thích'].values[i] = 0

    if pd.isna(cooked_df['yêu thích'].values[i]):
        cooked_df['yêu thích'].values[i] = 0

    if pd.isna(cooked_df['haha'].values[i]):
        cooked_df['haha'].values[i] = 0

    if pd.isna(cooked_df['wow'].values[i]):
        cooked_df['wow'].values[i] = 0

    if pd.isna(cooked_df['buồn'].values[i]):
        cooked_df['buồn'].values[i] = 0

    if pd.isna(cooked_df['phẫn nộ'].values[i]):
        cooked_df['phẫn nộ'].values[i] = 0

    if pd.isna(cooked_df['thương thương'].values[i]):
        cooked_df['thương thương'].values[i] = 0

    if pd.isna(cooked_df['shares'].values[i]):
        cooked_df['shares'].values[i] = 0
    
    if pd.isna(cooked_df['comments'].values[i]):
        cooked_df['comments'].values[i] = 0

cooked_df['thích'] = cooked_df['thích'].astype(int)
cooked_df['yêu thích'] = cooked_df['yêu thích'].astype(int)
cooked_df['haha'] = cooked_df['haha'].astype(int)
cooked_df['wow'] = cooked_df['wow'].astype(int)
cooked_df['thương thương'] = cooked_df['thương thương'].astype(int)
cooked_df['buồn'] = cooked_df['buồn'].astype(int)
cooked_df['phẫn nộ'] = cooked_df['phẫn nộ'].astype(int)

### 2.2.5 Thêm cột thể hiện giờ đăng bài viết

In [13]:
cooked_df['time'] = cooked_df['time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df = cooked_df['posted_hour'] = cooked_df['time'].apply(lambda x: x.hour)

## 2.3 Tổng kết

In [14]:
cooked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   post_url        123 non-null    object        
 1   post_text       123 non-null    object        
 2   time            123 non-null    datetime64[ns]
 3   image           91 non-null     object        
 4   video           12 non-null     object        
 5   comments        123 non-null    int64         
 6   shares          123 non-null    int64         
 7   user_id         123 non-null    int64         
 8   comments_full   123 non-null    object        
 9   reaction_count  123 non-null    int64         
 10  thích           123 non-null    int32         
 11  yêu thích       123 non-null    int32         
 12  haha            123 non-null    int32         
 13  wow             123 non-null    int32         
 14  thương thương   123 non-null    int32         
 15  buồn  

In [15]:
print(f"Kích cỡ dữ liệu thô : {raw_df.shape[0]} x {raw_df.shape[1]}")

Kích cỡ dữ liệu thô : 300 x 53


In [16]:
print(f"Kích cỡ dữ liệu đã được xử lí : {cooked_df.shape[0]} x {cooked_df.shape[1]}")

Kích cỡ dữ liệu đã được xử lí : 123 x 18


### 2.3.2 Lưu dữ liệu đã xử lí

In [17]:
cooked_df.to_csv('Data/GenshinImpact.vi_modified.csv', index=False)